<a href="https://colab.research.google.com/github/aerosense-ai/notebooks/blob/main/Information_sensors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!pip install --upgrade git+https://github.com/aerosense-ai/aerosense-tools.git@0.4.0

In [3]:
import datetime as dt

from aerosense_tools.queries import BigQuery
from aerosense_tools import plots
from aerosense_tools.preprocess import RawSignal
from aerosense_tools.preprocess import SensorMeasurementSession

client = BigQuery()

In [13]:
info_sensors_metadata = {
    "connection_statistics": {
        "description": "Connection Statistics",
        "variable": [
            "Transmitting Power [W]",
            "Allocated Heap Memory [Bytes]",
            "Received signal strength indication: Raw [dBm]",
            "Received signal strength indication: Filtered [dBm]"
        ],
        "sensors": ["Conn. Stat. {}".format(i) for i in range(4)]
    },
    "battery_info": {
        "description": "Battery State information",
        "variable": [
            "Voltage [V]",
            "Number of cycles",
            "Battery Capacity [%]"
        ],
        "sensors": ["Batt. info. {}".format(i) for i in range(3)]
    },
}

# Battery Information

In [8]:
node="1"
installation = "aventa-turbine-test"
measurement_day = dt.datetime(2023, 1, 1)
query_period = dt.timedelta(days=365)

df, data_limit_applied = client.get_sensor_data(
    installation_reference=installation,
    node_id=node,
    sensor_type_reference="battery_info",
    start=measurement_day,
    finish=measurement_day+query_period,
    row_limit=10000000,
)

data_columns =  df.columns[df.columns.str.startswith('f')].tolist()
info_df = df[["datetime"]+data_columns].set_index('datetime')
info_df.columns = info_sensors_metadata["battery_info"]["variable"]

In [12]:
import plotly.express as px

info_df.drop(["Number of cycles"], axis=1, inplace=True)

layout={
    "xaxis_title":"Date/Time"
}
for variable in info_df.columns:
  figure = px.line(info_df, y=variable)
  figure.update_layout(layout)
  figure.show()